In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dropout, Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Embedding, SpatialDropout1D, Attention


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import array
from tensorflow.keras.utils import to_categorical
# from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import  Dropout, Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

In [4]:
import os

directory = '/kaggle/input/nlp-modeling'
files = os.listdir(directory)
print(files)

['Data4Model.csv']


In [11]:
df = pd.read_csv(r'/kaggle/input/nlp-modeling/Data4Model.csv')
df.sample(10)

,preprocessing,dialect
23820,وين يدير الاي اشنابو,LY
41313,شكرا بزاف خويا مهدي,MA
61237,مفيش اي سيناريو مكن يخلي المصرين المفروض الفلس...,EG
76100,لسه كسبان جيمين الماب الجديده,EG
67648,عجبني اسم هاني رمزي ساعه وعيت علي الكره عرفته ...,EG
74928,يارب احشر حينتخب السيسي عارف بلاويه وعمايله مع...,EG
19761,درتي زمالطوك مزال,LY
61031,ياخويا الدنيا مكركبه عندهم غيري,EG
43187,طاحت الصمعه علقو الحجام,MA
69123,اله ينور كلامك كله صح فعلا محجوب عبد الدايم وا...,EG


In [12]:
df.isnull().sum()

preprocessing    92
dialect           0
dtype: int64

I found some NaNs after saving my cleaned/processed data, I don't know why and how to handle this problem 'until now', so I'm gonna drop NaNs because it's a small number

In [13]:
print("Number of missing values in each column:")
print(df.isnull().sum())

# Drop rows with missing values
df.dropna(inplace=True)

# Verify that NaNs have been dropped
print("Number of missing values after dropping rows:")
print(df.isnull().sum())

Number of missing values in each column:
preprocessing    92
dialect           0
dtype: int64
Number of missing values after dropping rows:
preprocessing    0
dialect          0
dtype: int64


In [14]:
df.sample(15)

,preprocessing,dialect
50653,محمد كريم اسكندريه,EG
48883,عمرك كام عرفت ان مدام انتصار تبقي بنت خاله السيسي,EG
35924,ضد اكيد شن علاقتهم الزب ناس تسترزق راسها,LY
62898,تكون مش فاهم حاجه الحصه ومش عارف تعمل ايه,EG
8311,اعتقد ان الخضر الراي,LY
64062,دي ارزاق ميزو,EG
79784,وانت بتاخد صرمك ورا ياعلق,EG
77452,الحاجه الي تحرق الدم ان اكتر ناس هتلاقيهم بيكت...,EG
52074,انا انت حبيبي مصطفي اوكسمبيلاه,EG
45068,خلاص مش هتبان حاجه دلوقتي,MA


In [16]:
from nltk.tokenize import word_tokenize


In [17]:
df['tokenizer'] = df['preprocessing'].apply(lambda x: word_tokenize(x))
df.sample(3)

,preprocessing,dialect,tokenizer
69280,يقفلوا حساب تفتح الف احنا وراك داءما باذن اله ...,EG,"[يقفلوا, حساب, تفتح, الف, احنا, وراك, داءما, ب..."
5895,وسط ميلان يمرض دينمك,LY,"[وسط, ميلان, يمرض, دينمك]"
41996,وراه اغنيه نيت,MA,"[وراه, اغنيه, نيت]"


In [25]:
df.rename(columns={'preprocessing': 'clean_text'}, inplace=True)

df.head()

,clean_text,dialect,tokenizer
0,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...,LY,"[قليلين, ادب, ومنافقين, اختهم, او, قريبتهم, تع..."
1,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...,LY,"[اليبين, متقلبين, بالنسبه, ليا, انا, ميليشياوي..."
2,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,LY,"[تانيه, شاب, ليبي, بيرتاح, لبنت, مختلفه, ويلاح..."
3,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,LY,"[رانيا, عقليتك, متخلفه, اولا, الانسان, يلي, يح..."
4,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,LY,"[شكلك, متعقده, علشان, الراجل, تحبيه, ازوج, بنت..."


In [39]:
df.drop(columns=['clean_text'], inplace=True)
df

,dialect,tokenizer
0,LY,"[قليلين, ادب, ومنافقين, اختهم, او, قريبتهم, تع..."
1,LY,"[اليبين, متقلبين, بالنسبه, ليا, انا, ميليشياوي..."
2,LY,"[تانيه, شاب, ليبي, بيرتاح, لبنت, مختلفه, ويلاح..."
3,LY,"[رانيا, عقليتك, متخلفه, اولا, الانسان, يلي, يح..."
4,LY,"[شكلك, متعقده, علشان, الراجل, تحبيه, ازوج, بنت..."
...,...,...
147720,SD,"[الناس, دي, بتنفخ, قربه, مقدوده, بالدارجي, كده..."
147721,SD,"[انت, عايش, وين, بره, السودان, شنو, ماشايف, ال..."
147722,SD,"[ماحرم, نفسي, ميسي, حريف, ولعاب, برضو, مدريدي,..."
147723,SD,"[ياخي, ديل, ماخلو, لشيطان, وابليس, شي, يروحو, ..."


In [41]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df.tokenizer.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 199625 unique tokens.


In [42]:
df

,dialect,tokenizer
0,LY,"[قليلين, ادب, ومنافقين, اختهم, او, قريبتهم, تع..."
1,LY,"[اليبين, متقلبين, بالنسبه, ليا, انا, ميليشياوي..."
2,LY,"[تانيه, شاب, ليبي, بيرتاح, لبنت, مختلفه, ويلاح..."
3,LY,"[رانيا, عقليتك, متخلفه, اولا, الانسان, يلي, يح..."
4,LY,"[شكلك, متعقده, علشان, الراجل, تحبيه, ازوج, بنت..."
...,...,...
147720,SD,"[الناس, دي, بتنفخ, قربه, مقدوده, بالدارجي, كده..."
147721,SD,"[انت, عايش, وين, بره, السودان, شنو, ماشايف, ال..."
147722,SD,"[ماحرم, نفسي, ميسي, حريف, ولعاب, برضو, مدريدي,..."
147723,SD,"[ياخي, ديل, ماخلو, لشيطان, وابليس, شي, يروحو, ..."


In [52]:
df['dialect'].unique()

array(['LY', 'MA', 'EG', 'LB', 'SD'], dtype=object)

In [43]:
X = tokenizer.texts_to_sequences(df.tokenizer.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (147633, 250)


In [44]:
Y = pd.get_dummies(df['dialect']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (147633, 5)


In [71]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [74]:
X = df['tokenizer']
y = df['dialect']

X = [' '.join(map(str, lst)) for lst in X]


In [75]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index

In [76]:
max_length = max(len(seq) for seq in X_seq)
X_padded = pad_sequences(X_seq, maxlen=max_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [77]:
vocab_size = len(word_index) + 1
embedding_dim = 128
lstm_units = 64
num_classes = 5  # Number of classes

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [85]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=2)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2953/2953 - 290s - 98ms/step - accuracy: 0.3914 - loss: 1.4548 - val_accuracy: 0.3869 - val_loss: 1.4523
Epoch 2/10
2953/2953 - 286s - 97ms/step - accuracy: 0.4201 - loss: 1.4072 - val_accuracy: 0.4243 - val_loss: 1.3928
Epoch 3/10
2953/2953 - 284s - 96ms/step - accuracy: 0.4207 - loss: 1.3887 - val_accuracy: 0.4134 - val_loss: 1.3853
Epoch 4/10
2953/2953 - 280s - 95ms/step - accuracy: 0.3917 - loss: 1.4035 - val_accuracy: 0.3869 - val_loss: 1.4066
Epoch 5/10
2953/2953 - 299s - 101ms/step - accuracy: 0.3917 - loss: 1.4246 - val_accuracy: 0.3869 - val_loss: 1.4345
Epoch 6/10
2953/2953 - 291s - 99ms/step - accuracy: 0.3915 - loss: 1.4274 - val_accuracy: 0.3869 - val_loss: 1.4377
Epoch 7/10
2953/2953 - 282s - 96ms/step - accuracy: 0.3915 - loss: 1.4337 - val_accuracy: 0.3869 - val_loss: 1.4351
Epoch 8/10
2953/2953 - 285s - 96ms/step - accuracy: 0.4528 - loss: 1.3205 - val_accuracy: 0.5439 - val_loss: 1.0947
Epoch 9/10
2953/2953 - 284s - 96ms/step - accuracy: 0.7258 - loss: 0.7556 - val_ac

In [87]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy}')


923/923 - 22s - 23ms/step - accuracy: 0.8106 - loss: 0.5633
Test Accuracy: 0.8106479048728943


In [99]:
model.save("DeepModelNLP.h5")

In [103]:
model.save(r'E:\NLP\Final Project\Ahmed_Mostafa_attia_NLP_Project\DL\DeepLModelNLP.h5')


In [90]:
new_data = [
    "الناس دي بتنفخ قربه مقدوده بالدارجي كده البلد دي ماعندهاش حل",  # Example from SD
    "رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل ليشوف مصلحته ضعيف شخصية",  # Example from LY
]


In [98]:
import numpy as np

arabic_texts = ["ازيك يسطا", "ياعم اشطا ", "كيف حالك أخي؟"]


sequences = tokenizer.texts_to_sequences(arabic_texts)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

predictions = model.predict(padded_sequences)

original_labels = label_encoder.inverse_transform(predicted_classes)

print(original_labels)
predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
['EG' 'EG' 'LB']
[0 0 1]


In [96]:
y_train

array([1, 3, 0, ..., 1, 4, 1])

In [105]:
from tensorflow.keras.models import load_model 

In [106]:
loaded_model = load_model('DeepModelNLP.h5')

In [107]:
loaded_model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)        │ (None, 61, 128)        │    25,552,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_14            │ (None, 61, 128)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,601,735 (97.66 MB)

 Trainable params: 25,601,733 (97.66 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)